<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [2]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [3]:
def magField(current,distance,xy_points,mu):
  if type(xy_points)!=np.ndarray:
    field = (mu*current)/(2*np.pi*distance)
  else:
    field = np.ones((xy_points.shape[1],xy_points.shape[2]))
    field_above=[]; field_below=[]; coordinate_below =[]; coordinate_above =[]

    for i in range(xy_points.shape[1]):
      for j in range(xy_points.shape[2]):
        x=xy_points[:,i,j]
        if (x[0]==0 and x[1]==0):
          field[i,j] = None
        else:
          if x[1]<0:
            mu_below = mu*(1-(2.2e-5))
            field[i,j] = (mu_below*current)/(2*np.pi*np.sqrt(x[0]**2+x[1]**2))
            field_below.append(field[i,j])
            coordinate_below.append(x)
          elif x[1]>=0:
            field[i,j] = (mu*current)/(2*np.pi*np.sqrt(x[0]**2+x[1]**2))
            field_above.append(field[i,j])
            coordinate_above.append(x)

    print(f' Coordinate Below : {coordinate_below[-xy_points.shape[1]:]}')
    print(f' Field Below : {field_below[-xy_points.shape[1]:]}')
    print(f' Coordinate Above 0 : {coordinate_above[xy_points.shape[1]-1:2*xy_points.shape[1]-1]}')
    print(f' Field Above 0 : {field_above[xy_points.shape[1]:2*xy_points.shape[1]]}')
    print(f'\n Field Comparison for coordinates between (-y and 0) : \n{np.array(field_below[-xy_points.shape[1]:])<np.array(field_above[0:xy_points.shape[1]])}')
    print(f'Field Comparison for coordinates between (-y and y) : \n{np.array(field_below[-xy_points.shape[1]:])<np.array(field_above[xy_points.shape[1]-1:2*xy_points.shape[1]-1])}')
  return field

In [4]:
def circle(circle_points,line_points,mu):

  x_array=[]; y_array=[]
  for i,(points, _points) in enumerate(zip(line_points,circle_points)):
    x_array_points = []; y_array_points = []
    array_points = np.array(_points)
    below_pi = array_points[np.where((array_points<=np.pi))[0]]
    above_pi = array_points[np.where((array_points>np.pi))[0]]
    if mu==None:
      # print('It is for Coordinate.')
      x_array_points.append(np.concatenate((points*np.cos(below_pi),points*np.cos(above_pi))))
      y_array_points.append(np.concatenate((points*np.sin(below_pi),points*np.sin(above_pi))))
    else:
      mu_below = mu*(1-(2.2e-5))
      # print('It is for Magnetic field.')
      x_array_points.append(np.concatenate((points*np.cos(below_pi),points*(mu_below/mu)*np.cos(above_pi))))
      y_array_points.append(np.concatenate((points*np.sin(below_pi),points*(mu_below/mu)*np.sin(above_pi))))

      # if ((points*np.sin(below_pi))[0]>(points*(mu_below/mu)*np.sin(above_pi))[-1])==False:
      #   print((points*np.sin(below_pi))[0],(points*(mu_below/mu)*np.sin(above_pi))[-1])
      print(f' Left Edge : {(points*np.sin(below_pi))[-1]>(points*(mu_below/mu)*np.sin(above_pi))[0]}\t \
      Right Edge : {(points*np.sin(below_pi))[0]>(points*(mu_below/mu)*np.sin(above_pi))[-1]}')
    x_array.append(x_array_points); y_array.append(y_array_points)
  return x_array, y_array

In [5]:
wire_radius = 25e-6; distance = 50e-6; wire_current = 10; mu_freeSpace = 4*np.pi*1e-7
linear_density = 10; init_angle = 0; final_angle = 2*np.pi
min_circular_density = 10; point_addition = 1

linear_points = np.linspace(wire_radius, distance, linear_density); inside_points = 0*np.linspace(0,wire_radius,10)
total_distance = np.concatenate((inside_points,linear_points))

circular_points=[]; circular_angles=[]
[circular_points.append(int(min_circular_density + circular_density*point_addition)) for circular_density in range(total_distance.shape[0])]
circular_points = np.array(circular_points)
if np.all(circular_points)==False:
  raise Exception('Check the values of circular_points')
circular_angles = [np.linspace(init_angle, final_angle, _points, endpoint=False) for _points in circular_points]

mag_field = magField(current=wire_current, xy_points=False,distance=linear_points,mu=mu_freeSpace)
x_points, y_points = circle(mu=None,circle_points=circular_angles,line_points=total_distance)

In [6]:
x_linear_point = np.concatenate((-linear_points[::-1],inside_points,linear_points))
y_linear_point = x_linear_point

In [28]:
def MagField_Circle(current,mu,wire_radius,x,y):
  field = np.ones((x.shape[0],y.shape[0]))
  field_xy = np.ones((x.shape[0],y.shape[0],2))
  quiver_xy = np.ones((x.shape[0],y.shape[0],2))
  for i in range(x.shape[0]):
    for j in range(y.shape[0]):
      radius = np.sqrt(x[i]**2+y[j]**2)
      if radius<=wire_radius:
        field[i,j] = None
        field_xy[i,j] = np.array([None,None])
        quiver_xy[i,j] = np.array([x[i],y[j]])
      else:
        if y[j]<=(-wire_radius):
          if x[i]!=0:
            angle = np.arctan(y[j]/x[i])
          else:
            angle = np.arctan(y[j]*1e18)
          mu_below = mu*(1-(2.2e-5))
          field[i,j] = (mu_below*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy[i,j] = np.array([field[i,j]*np.cos(angle), field[i,j]*np.sin(angle)])
          quiver_xy[i,j] = np.array([x[i],y[j]])
        else:
          if x[i]!=0:
            angle = np.arctan(y[j]/x[i])
          else:
            angle = np.arctan(y[j]*1e18)
          field[i,j] = (mu*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy[i,j] = np.array([field[i,j]*np.cos(angle), field[i,j]*np.sin(angle)])
          quiver_xy[i,j] = np.array([x[i],y[j]])
  return field,field_xy,quiver_xy

In [29]:
mag_field,field_xy,quiver_xy = MagField_Circle(current=wire_current,mu=mu_freeSpace,wire_radius=wire_radius,x=x_linear_point,y=y_linear_point)

In [30]:
mag_field.shape,field_xy.shape,quiver_xy.shape

((30, 30), (30, 30, 2), (30, 30, 2))

In [45]:
quiver_xy[0,:,:],field_xy[0,:,:]

(array([[-5.00000000e-05, -5.00000000e-05],
        [-5.00000000e-05, -4.72222222e-05],
        [-5.00000000e-05, -4.44444444e-05],
        [-5.00000000e-05, -4.16666667e-05],
        [-5.00000000e-05, -3.88888889e-05],
        [-5.00000000e-05, -3.61111111e-05],
        [-5.00000000e-05, -3.33333333e-05],
        [-5.00000000e-05, -3.05555556e-05],
        [-5.00000000e-05, -2.77777778e-05],
        [-5.00000000e-05, -2.50000000e-05],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  0.00000000e+00],
        [-5.00000000e-05,  2.50000000e-05],
        [-5.00000000e-05,  2.77777778e-05],
        [-5.00000000e-05,  3.055

In [48]:
fig = go.Figure()
# fig.add_trace(go.Contour(x=x_linear_point,y=y_linear_point,z=mag_field,contours_coloring='heatmap',
#                                 connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
# fig.update_traces(line_width=0)

# for i in range(0,quiver_x.shape[0]):
#   for j in range(0,quiver_y.shape[0]):
#     x = np.array([quiver_x[i,j]]); y = np.array([quiver_y[i,j]])
#     u = [-mag_field[i,j]*np.sin(np.arctan(y/x))]; v = [mag_field[i,j]*np.cos(np.arctan(y/x))]
#     fig1 = ff.create_quiver(x, y, u, v, scale=0.0001, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))
#     fig.add_traces(data=fig1.data)
for i in range(quiver_xy.shape[0]):
    x = quiver_xy[i,:,0]; y = quiver_xy[i,:,1]
    u = -field_xy[i,:,1]; v = field_xy[i,:,0]
    quiver_trace = ff.create_quiver(x, y, u, v, scale=1, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black')).data[0]
    fig.add_trace(quiver_trace)

fig.update_layout(template=fig_template, title = None,width=800, height=700)

fig.show()

In [10]:
Bnew = magField(current=wire_current,xy_points=False, distance=total_distance,mu=mu_freeSpace)
Bnew[np.where(np.isinf(Bnew))[0]]=0
B_x,B_y = circle(circular_angles,mu=mu_freeSpace,line_points=Bnew)

 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : False	       Right Edge : False
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True
 Left Edge : True	       Right Edge : True


<ipython-input-3-d98664ab0d9c>:3: RuntimeWarning:

divide by zero encountered in divide



In [11]:
x_linear_point = np.concatenate((-linear_points[::-1],inside_points,linear_points))
y_linear_point = x_linear_point

X, Y = np.meshgrid(x_linear_point, y_linear_point)
# r=np.sqrt(X**2+Y**2)

B = magField(current=wire_current, distance=None, mu= mu_freeSpace,xy_points=np.array([X,Y]))
# B = magField(current=wire_current, distance=r, mu= mu_freeSpace,xy_points=False)
# B = (mu_freeSpace*wire_current)/(2*np.pi*r)

 Coordinate Below : [array([-5.0e-05, -2.5e-05]), array([-4.72222222e-05, -2.50000000e-05]), array([-4.44444444e-05, -2.50000000e-05]), array([-4.16666667e-05, -2.50000000e-05]), array([-3.88888889e-05, -2.50000000e-05]), array([-3.61111111e-05, -2.50000000e-05]), array([-3.33333333e-05, -2.50000000e-05]), array([-3.05555556e-05, -2.50000000e-05]), array([-2.77777778e-05, -2.50000000e-05]), array([-2.5e-05, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 0.0e+00, -2.5e-05]), array([ 2.5e-05, -2.5e-05]), array([ 2.77777778e-05, -2.50000000e-05]), array([ 3.05555556e-05, -2.50000000e-05]), array([ 3.33333333e-05, -2.50000000e-05]), array([ 3.61111111e-05, -2.50000000e-05]), array([ 3.88888889e-05, -2.50000000e-05]), array([ 4.16666667e-05, -2.50000000e-05]), 

In [12]:
x_points[1][0]

array([ 0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.])

In [13]:
fig = go.Figure()
fig.add_trace(go.Contour(x=x_linear_point,y=y_linear_point+wire_radius,z=B,contours_coloring='heatmap',
                                connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
fig.update_traces(line_width=0)
fig.add_trace(go.Scatter(x=wire_radius*np.cos(np.linspace(0,2*np.pi,1000)), y=wire_radius + wire_radius*np.sin(np.linspace(0,2*np.pi,1000)),
                         mode='lines', line=dict(color='yellow', width=0.7),fill = 'tonexty',fillcolor = 'rgba(127, 0, 255, 0.5)'))

for radi in range(0,total_distance.shape[0],10):
    x = x_points[radi][0]; y = y_points[radi][0] + wire_radius
    u = -B_y[radi][0]; v = B_x[radi][0]
    fig1 = ff.create_quiver(x, y, u, v, scale=0.0001, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))
    fig.add_traces(data=fig1.data)

fig.add_trace(go.Scatter(x=[0], y=[wire_radius], mode='markers', marker_size=15, marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=x_linear_point, y=0*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
fig.add_trace(go.Scatter(x=x_linear_point, y=(-distance+wire_radius)*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='grey'),
                         fill='tonexty',fillcolor = 'rgba(255, 255, 255, 0.5)'))

fig.update_layout(showlegend=False)
fig.update_layout(template=fig_template, title = None,width=800, height=700)
fig.update_xaxes(title = 'Distance',range = [-distance,distance]); fig.update_yaxes(title = 'Distance', range = np.array([-distance,distance])+wire_radius)
# fig.write_html('MagField_RadDistance.html')
fig.show()